In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.ticker as mtick

In [19]:
base_path = os.path.join(os.getcwd(), '..', 'steam-insights-main')

games_df_path = os.path.join(base_path, 'games.csv')
genres_path = os.path.join(base_path, 'genres.csv')
steamspy_insights_path = os.path.join(base_path, 'steamspy_insights.csv')
tags_path = os.path.join(base_path, 'tags.csv')
reviews_path = os.path.join(base_path, 'reviews.csv')

In [33]:
# Read games.csv (only app_id and name) and drop rows with missing name
games = pd.read_csv(
    games_df_path,
    usecols=["app_id", "name", "release_date"],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    delimiter=',',
    engine='python',
    on_bad_lines='skip'
)
games = games.dropna(subset=['name'])


# Price and appID from steamspy
steamspy = pd.read_csv(
    steamspy_insights_path,
    usecols=["app_id", "price"],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    delimiter=',',
    engine='python',
    on_bad_lines='skip',
    na_values=["\\N"]
)
# Immediately drop rows with missing values in 'price'
steamspy = steamspy.dropna(subset=['price'])
# Replace 'N' in the 'price' column with NaN
steamspy['price'] = steamspy['price'].replace('N', np.nan)
# Convert price to numeric and handle any invalid entries as NaN
steamspy['price'] = pd.to_numeric(steamspy['price'], errors='coerce')
# Drop rows with NaN in 'price' after conversion
steamspy = steamspy.dropna(subset=['price'])
# Adjust price (divide by 100 to get price in euros)
steamspy['price'] = steamspy['price'] / 100



# Read tags.csv (only app_id and tag) and drop rows with missing tag - might need to group by appID to collect all tags together
tags_df = pd.read_csv(
    tags_path,
    usecols=["app_id", "tag"],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    delimiter=',',
    engine='python',
    on_bad_lines='skip',
    na_values=["\\N"]
)
tags_df = tags_df.dropna(subset=['tag'])


# Read genres.csv (only app_id and genre) and drop rows with missing genre
genres_df = pd.read_csv(
    genres_path,
    usecols=["app_id", "genre"],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    delimiter=',',
    engine='python',
    on_bad_lines='skip',
    na_values=["\\N"]
)
genres_df = genres_df.dropna(subset=['genre'])
# Group tags by app_id, joining multiple tags into one string
tags = tags_df.groupby("app_id")["tag"].apply(lambda x: ", ".join(x.astype(str))).reset_index()
# Group genres by app_id, joining multiple genres into one string
genres = genres_df.groupby("app_id")["genre"].apply(lambda x: ", ".join(x.astype(str))).reset_index()



# Read reviews.csv and drop rows with missing genre
reviews = pd.read_csv(
    reviews_path,
    usecols=["app_id", "review_score", "positive", "negative", "total"],
    quotechar='"',
    doublequote=True,
    escapechar='\\',
    delimiter=',',
    engine='python',
    on_bad_lines='skip',
    na_values=["\\N"]
)
reviews = reviews.dropna(subset=['review_score', 'positive', 'negative', 'total'])


# Shows the number of rows in each data set
print("games:", len(games.axes[0]), "rows and", len(games.axes[1]), "columns")
print("steamspy:", len(steamspy.axes[0]), "rows and", len(steamspy.axes[1]), "columns")
print("tags:", len(tags.axes[0]), "rows and", len(tags.axes[1]), "columns")
print("genres:", len(genres.axes[0]), "rows and", len(genres.axes[1]), "columns")
print("reviews:", len(reviews.axes[0]), "rows and", len(reviews.axes[1]), "columns")

games: 140082 rows and 3 columns
steamspy: 90890 rows and 2 columns
tags: 117505 rows and 2 columns
genres: 122458 rows and 2 columns
reviews: 140082 rows and 5 columns


In [34]:
# Shows the tags are grouped for each game now
display(tags.head(10))
display(genres.head(10))

,app_id,tag
0,10,"1980s, 1990's, Action, Assassin, Classic, Comp..."
1,20,"1990's, Action, Class-Based, Classic, Co-op, C..."
2,30,"Action, Class-Based, Classic, Co-op, Difficult..."
3,40,"1990's, Action, Arena Shooter, Classic, Co-op,..."
4,50,"1990's, Action, Adventure, Aliens, Atmospheric..."
5,60,"3D, Action, Classic, Competitive, Conspiracy, ..."
6,70,"1990's, Action, Action-Adventure, Adventure, A..."
7,80,"Action, Adventure, Atmospheric, Classic, Compe..."
8,130,"1990's, Action, Adventure, Aliens, Atmospheric..."
9,220,"Action, Adventure, Aliens, Atmospheric, Classi..."


,app_id,genre
0,10,Action
1,20,Action
2,30,Action
3,40,Action
4,50,Action
5,60,Action
6,70,Action
7,80,Action
8,130,Action
9,219,Action


In [ ]:
# Merge games and steamspy with an inner join so that only games present in both are kept
df_merged = pd.merge(games, steamspy, on="app_id", how="inner")

# Merge with grouped tags and genres (left join so that if a game lacks tags/genres, it still remains)
df_merged = pd.merge(df_merged, tags, on="app_id", how="left")
df_merged = pd.merge(df_merged, genres, on="app_id", how="left")

# Fill missing values in tag and genre with default values
df_merged['tag'] = df_merged['tag'].fillna('No Tags')
df_merged['genre'] = df_merged['genre'].fillna('No Genre')

df_merged = pd.merge(df_merged, reviews, on="app_id", how="inner")


missing_date = df_merged['release_date'].isna().sum()
missing_price = df_merged['price'].isna().sum()
missing_tag = df_merged['tag'].isna().sum()
missing_genre = df_merged['genre'].isna().sum()
missing_review = df_merged['review_score'].isna().sum()
missing_pos = df_merged['positive'].isna().sum()
missing_neg = df_merged['negative'].isna().sum()
missing_total = df_merged['total'].isna().sum()

print(f"Missing 'missing_date' values: {missing_date}")
print(f"Missing 'price' values: {missing_price}")
print(f"Missing 'price' values: {missing_tag}")
print(f"Missing 'price' values: {missing_genre}")
print(f"Missing 'price' values: {missing_review}")
print(f"Missing 'price' values: {missing_pos}")
print(f"Missing 'price' values: {missing_neg}")
print(f"Missing 'price' values: {missing_total}")



display(df_merged.head())
print(f"Total rows in merged data: {df_merged.shape[0]}")

,app_id,name,release_date,price,tag,genre,review_score,positive,negative,total
0,10,Counter-Strike,2000-11-01,9.99,"1980s, 1990's, Action, Assassin, Classic, Comp...",Action,9,235403,6207,241610
1,20,Team Fortress Classic,1999-04-01,4.99,"1990's, Action, Class-Based, Classic, Co-op, C...",Action,8,7315,1094,8409
2,30,Day of Defeat,2003-05-01,4.99,"Action, Class-Based, Classic, Co-op, Difficult...",Action,8,6249,672,6921
3,40,Deathmatch Classic,2001-06-01,4.99,"1990's, Action, Arena Shooter, Classic, Co-op,...",Action,8,2542,524,3066
4,50,Half-Life: Opposing Force,1999-11-01,4.99,"1990's, Action, Adventure, Aliens, Atmospheric...",Action,9,22263,1111,23374


Total rows in merged data: 90890
